In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import brentq
from ipywidgets import interact, FloatSlider
import ipywidgets as widgets

In [ ]:
# --- Helper Functions ---

def H(p):
    p = np.asarray(p)
    res = np.zeros_like(p, dtype=float)
    mask = (p > 0) & (p < 1)
    res[mask] = -p[mask] * np.log2(p[mask]) - (1 - p[mask]) * np.log2(1 - p[mask])
    res[p <= 0] = 0
    res[p >= 1] = 0
    if res.size == 1: return res.item()
    return res

# --- Equation Definitions ---
def d1_derivative(d1, d0, delta, A, B):
    try:
        if d1 <= d0*A or d1 >= 1 - d0*B: return np.nan
        if d1 * A > delta: return np.nan
        
        term1 = -np.log2(1 - (d0 * A) / d1)
        term2 = np.log2(1 - (d0 * B) / (1 - d1))
        term3 = A * np.log2(delta / (d1 * A) - 1)
        term4 = B * np.log2((1 - delta) / (d1 * B) - 1)
        term5 = A * H(B/A)
        term6 = np.log2(d1 / (1 - d1))
        
        return term1 + term2 + term3 + term4 + term5 + term6
    except:
        return np.nan


def d0_derivative(d0, d1, r, A, B):
    try:
        if d0 >= 1: return np.nan
        if d1 <= d0 * A: return np.nan
        if (1 - d1) <= d0 * B: return np.nan

        term1 = (1.0/r - 1) * np.log2(1.0/d0 - 1)
        term2 = A * H(B/A)
        term3 = A * np.log2(d1 / (d0 * A) - 1)
        term4 = B * np.log2((1 - d1) / (d0 * B) - 1)
        
        return term1 + term2 + term3 + term4
    except:
        return np.nan


def exponent_equation(d0, d1, delta, r, A, B, n=1):
    try:
        t1 = (1.0/r) * H(d0)
        t2 = d0 * A * H(B/A)
        t3 = d1 * H((d0 * A) / d1)
        t4 = (1 - d1) * H((d0 * B) / (1 - d1))
        t5 = -H(d0)
        t6 = d1 * A * H(B/A)
        t7 = delta * H((d1 * A) / delta)
        t8 = (1 - delta) * H((d1 * B) / (1 - delta))
        t9 = -H(d1)
        return t1 + t2 + t3 + t4 + t5 + t6 + t7 + t8 + t9
    except:
        return np.nan


def exponent_equation_just_d0(d0, d1, delta, r, A, B):
    try:
        t1 = (1.0/r) * H(d0)
        t2 = d0 * A * H(B/A)
        t3 = d1 * H((d0 * A) / d1)
        t4 = (1 - d1) * H((d0 * B) / (1 - d1))
        t5 = -H(d0)
        t6 = d1 * A * H(B/A)
        t9 = -H(d1)
        return t1 + t2 + t3 + t4 + t5 + t6 + t9
    except:
        return np.nan


def exponent_equation_just_delta1(d0, d1, delta, r, A, B):
    try:
        t7 = delta * H((d1 * A) / delta)
        t8 = (1 - delta) * H((d1 * B) / (1 - delta))
        return t7 + t8
    except:
        return np.nan

In [ ]:
def plot_system(delta_val, c_val, r_val):
    if r_val == 0: return
    A = 1 - (c_val / r_val)
    B = c_val / r_val
    
    if B/A < 0 or B/A > 1:
        print(f"Error: c/r must be <= 0.5 (Currently B/A = {B/A:.2f})")
        return

    d1_vals = np.linspace(0.001, delta_val/A, 100)
    d1_x = []
    
    p1_val, p2_val, p3_val, p4_val = [], [], [], []
    
    for d1 in d1_vals:
        limit_a = 1.0
        limit_b = d1 / A
        limit_c = (1 - d1) / B
        upper_bound = min(limit_a, limit_b, limit_c) - 1e-6
        
        if upper_bound > 1e-6:
            try:
                d0_sol = brentq(d0_derivative, 1e-6, upper_bound, args=(d1, r_val, A, B))
                val_exp = exponent_equation(d0_sol, d1, delta_val, r_val, A, B)
                val_exp_2 = exponent_equation_just_d0(d0_sol, d1, delta_val, r_val, A, B)
                val_exp_3 = exponent_equation_just_delta1(d0_sol, d1, delta_val, r_val, A, B)
                d1_x.append(d1)
                p1_val.append(d0_sol)
                p2_val.append(val_exp)
                p3_val.append(val_exp_2)
                p4_val.append(val_exp_3)
            except:
                pass

    # --- Plotting ---
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
    
    # Plot 1
    ax1.plot(d1_x, p1_val, 'b-', lw=2)
    ax1.set_title(r"value of optimal d0")
    ax1.set_xlabel(r"$d_1$")
    ax1.set_ylabel(r"$d_0^*$")
    ax1.grid(True, alpha=0.3)
    
    # Plot 2
    ax2.plot(d1_x, p2_val, 'r-', lw=2)
    ax2.set_title(r"Exponent value")
    ax2.set_xlabel(r"$d_1$")
    ax2.set_ylabel("Exponent")
    ax2.grid(True, alpha=0.3)
    
    # Plot 3
    ax3.plot(d1_x, p3_val, 'g-', lw=2)
    ax3.set_title(r"Exponent value misc terms")
    ax3.set_xlabel(r"$d_1$")
    ax3.set_ylabel("Exp misc terms")
    ax3.grid(True, alpha=0.3)
    
    # Plot 4
    ax4.plot(d1_x, p4_val, 'purple', lw=2)
    ax4.set_title(r"Exponent value just delta terms")
    ax4.set_xlabel(r"$d_1$")
    ax4.set_ylabel("Exp delta terms")
    ax4.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

# Interactive Widgets
interact(plot_system, 
         delta_val=FloatSlider(min=0.01, max=0.99, step=0.001, value=0.1, description='delta (δ)'),
         c_val=FloatSlider(min=1.0, max=2.0, step=0.001, value=1.0, description='c'),
         r_val=FloatSlider(min=4, max=50, step=1, value=4, description='r'));